#A.I.N.A. 1.55

####**Artificially Intelligent News Anchor (A.I.N.A.)** is an AI that can get current news, summarize them, and tell them to you. The inspiration behind AINA came from my desire to keep up with financial news but the disinterest in browsing different sources. Additionally, when I tried new sources or podcasts that spoke about financial news they did not give me the types of information I desired. So, I decided to create AINA, a way to to quickly consume financial news in one place.

**Version 1.55 updates:**

This version introduces a **new summarization function** with the ability of taking in any article regardless of size/length and summarizing its content. AINA 1.55 also fixes some of the errors/bugs from previous version(s).

Fixes errors such as:
  * Actual vs. Target
  * Summarization function breaking
  * Latest funnction `while loop`






---









##Constructing A.I.N.A.

In [ ]:
!pip install gtts

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf

from copy import copy

import requests

# Web scraping
from bs4 import BeautifulSoup

#Transformer
from transformers import AutoTokenizer, pipeline

# Natural Language ToolKit
import nltk

# Audio
from gtts import gTTS
from IPython.display import Audio

In [ ]:
class AINA:

  def __init__(self):
    pass

#-------------------------------------------------------------------------------------------------#

  def get_story(self, urls):

    titles = []
    body = []

    for index, url in enumerate(urls):
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

        response = requests.get(url, headers=headers)

        # Fetch the article
        html_content = response.text

        # Parse the HTML content
        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract the title of the article by looking for the h1 tag
        title_tag = soup.find('title') #title instead of h1
        if title_tag:
            titles.append(title_tag.get_text())
        else:
            titles.append("No title found")

        # Extract the main content of the article
        article_body = soup.find_all('p')
        body_content = [paragraph.get_text() for paragraph in article_body[:-3]]
        body.append(' '.join(body_content) if body_content else "No content found")

    return titles, body

#-------------------------------------------------------------------------------------------------#

  def latest_news(self, latest=True):
    article_location = 0

    if not latest:
        return [], [], []

    # Define headers
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # Get the latest news page
    response = requests.get('https://finance.yahoo.com/topic/latest-news/', headers=headers)
    if response.status_code != 200:
        return [], [], []

    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract relevant article links
    article_links = [a.get('href') for a in soup.find_all('a', href=True) if '/news/' in a.get('href') and '.html' in a.get('href')]
    unique_links = []
    for link in article_links:
      if link not in unique_links:
        unique_links.append(link)
    for l in unique_links:
      if l[:4] != 'http':
       unique_links[unique_links.index(l)] = 'https://finance.yahoo.com' + l

    # Fetch article details
    titles, bodies = [], []
    for url in unique_links:
        article_response = requests.get(url, headers=headers)
        if article_response.status_code != 200:
            titles.append("No title found")
            bodies.append("No content found")
            continue

        article_soup = BeautifulSoup(article_response.text, 'html.parser')

        # Extract title
        title_tag = article_soup.find('title')
        titles.append(title_tag.get_text() if title_tag else "No title found")

        # Extract body content
        article_body = article_soup.find_all('p')
        body_content = [paragraph.get_text() for paragraph in article_body[:-3]]
        bodies.append(' '.join(body_content) if body_content else "No content found")


    search = 'No content found'
    body = bodies[article_location]

    while body == search:
      article_location += 1
      body = bodies[article_location]
      if body != search:
        break

    return [unique_links[article_location]], [titles[article_location]], [body]

#-------------------------------------------------------------------------------------------------#

  def general_news(self, article_amount):
    links = []

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    response = requests.get('https://www.yahoo.com/news/', headers=headers)
    if response.status_code != 200:
        return [], [], []

    # Fetch the article
    html_content = response.text

    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract the main content of the article
    article_body = soup.find_all('a', href=True)

    # Filter out the relevant links
    article_links = [link.get('href') for link in article_body if '/news/' in link.get('href')]# this is the source of the problems
    specific_links = ['https://www.yahoo.com/news/' + link for link in article_links if '.html' in link]

    # Remove duplicates
    # links = list(set(specific_links))
    links=[]
    for link in specific_links:
      if link not in links:
        links.append(link)
    # Limit to the requested number of articles
    links = links[:article_amount]

    # Fetch the articles and their titles
    titles = []
    body = []

    for url in links:
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            titles.append("No title found")
            body.append("No content found")
            continue

        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract the title of the article
        title_tag = soup.find('title')
        if title_tag:
            titles.append(title_tag.get_text())
        else:
            titles.append("No title found")

        # Extract the main content of the article
        article_body = soup.find_all('p')
        body_content = [paragraph.get_text() for paragraph in article_body]
        body.append(' '.join(body_content) if body_content else "No content found")

    return links, titles, body

#-------------------------------------------------------------------------------------------------#

  def related_news(self, tickers_list):

    links = []
    titles = []
    publisher = []

    # Collect news articles
    for tick in tickers_list:
        news = yf.Ticker(tick).news
        if news:
            links.append(news[0]['link'])
            titles.append(news[0]['title'])
            publisher.append(news[0]['publisher'])

    if len(tickers_list) > 1:
      title_list = titles.copy()
      link_list = links.copy()
      publisher_list = publisher.copy()

      actual_title = list(set(titles))  # Get unique titles

      search_dup = 'Duplicate'
      target = len(tickers_list)
      actual = len(actual_title)

      article_loc = 1

      while actual < target:
          title_list_copy = title_list.copy()

          for i in range(len(titles)):
              if len(title_list_copy) > 0:
                  title_list_copy.pop(0)
              else:
                  break
              if titles[i] in title_list_copy:
                  title_list[i] = search_dup

          find_dup = np.where(np.array(title_list) == search_dup)  # Find duplicates
          while search_dup in title_list:
              for i in find_dup[0]:
                  try:
                      news = yf.Ticker(tickers_list[i]).news
                      if len(news) > article_loc:
                          title_list[i] = news[article_loc]['title']
                          link_list[i] = news[article_loc]['link']
                          publisher_list[i] = news[article_loc]['publisher']
                  except IndexError:
                      title_list[i] = 'No more articles'
                  except Exception as e:
                      print(f"Error fetching news for {tickers_list[i]}: {e}")

          article_loc += 1
          actual = len(set(title_list))  # Recalculate unique titles
      links = link_list
      titles = title_list
      publisher = publisher_list

  ##------------------------------------------------------------------------------##

      ### This is to get the stories and titles from the links ###
      title_story = {}
      titles = []
      body = []

    for index, url in enumerate(links):
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

        response = requests.get(url, headers=headers)

        # Fetch the article
        html_content = response.text

        # Parse the HTML content
        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract the title of the article by looking for the h1 tag
        title_tag = soup.find('title') #title instead of h1
        if title_tag:
            titles.append(title_tag.get_text())
        else:
            titles.append("No title found")

        # Extract the main content of the article
        article_body = soup.find_all('p')
        body_content = [paragraph.get_text() for paragraph in article_body[:-3]]
        body.append(' '.join(body_content) if body_content else "No content found")

        title_story[titles[index]] = body[index]

  ##------------------------------------------------------------------------------##
  # This is to search for 'No content found' and replace it with an actual article

    search = 'No content found'
    target1 = len(tickers_list)
    actual1 = len([i for i in body if i != search])

    # Create a list in the event 'NO content found' is in body
    if actual1 < target1:
      body_copy = body.copy()
      title_copy = titles.copy()
      link_copy = links.copy()

      article_position = article_loc + 1 ## Can start from article_loc from previous if statement

      while actual1 < target1:
        finder1 = np.where(np.char.startswith(body, search)) # Find where the 'NCF' is/are
        for i in finder1[0]:
          link = [yf.Ticker(tickers_list[i]).news[article_position]['link']]
          title_story, body_story = self.get_story(link)
          body_copy[i] = body_story[0]
          title_copy[i] = title_story[0]
          link_copy[i] = link[0]

        article_position += 1
        actual1 = len([i for i in body_copy if i != search])
        print(f'Target: {target1} | Actual: {actual1}')

      body = body_copy
      titles = title_copy
      links = link_copy
    else:
      pass
    return links, titles, body

#-------------------------------------------------------------------------------------------------#

  def call_news_functions(self, general_amt, tickers_list, latest = True):
    url_specific, title_specific, body_specific = self.latest_news(latest)
    url_news, title_news, body_news = self.general_news(general_amt)
    link_related, title_related, body_related = self.related_news(tickers_list)

    title_story = {}

    link = []
    [link.extend(i) for i in [url_specific, url_news, link_related]]

    body = []
    [body.extend(i) for i in [body_specific, body_news, body_related]]

    title = []
    [title.extend(i) for i in [title_specific, title_news, title_related]]

    for i in range(len(link)):
      title_story[title[i]] = body[i]

    return link, title_story

#-------------------------------------------------------------------------------------------------#

  def summarize(self, story_dict):
    nltk.download('punkt')

    checkpoint = 'facebook/bart-large-cnn'

    tokenizer = AutoTokenizer.from_pretrained(checkpoint)

    summary = {}

    try:
      for title, story in story_dict.items():
        sentences = nltk.tokenize.sent_tokenize(story)


        length = 0
        chunk = ""
        chunks = []
        count = -1
        context = 1024
        for sentence in sentences:
          count += 1
          combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

          if combined_length  <= context: # if it doesn't exceed
            chunk += sentence + " " # add the sentence to the chunk
            length = combined_length # update the length counter

            # if it is the last sentence
            if count == len(sentences) - 1:
              chunks.append(chunk.strip()) # save the chunk

          else:
            chunks.append(chunk.strip()) # save the chunk

            # reset
            length = 0
            chunk = ""

            # take care of the overflow sentence
            chunk += sentence + " "
            length = len(tokenizer.tokenize(sentence))
        sum = ''
        for chunk in chunks:
          summarizer = pipeline("summarization", model = checkpoint)
          sum += summarizer(chunk, min_length=100, max_length=125)[0]['summary_text']
        summary[title] = sum
    except Exception as e:
      print(f'Could not produce summary: {e}')
    return summary

#-------------------------------------------------------------------------------------------------#

  def sanitize_filename(self, title): # This get's rid of any special charactes in the title allowing the file to be saved.
    # Replace invalid characters with underscores
    return ''.join(c if c.isalnum() or c in (' ', '_') else '_' for c in title)

#-------------------------------------------------------------------------------------------------#

  def voice(self, summarized):
    # Iterate over the dictionary and convert each story to speech
    for title, story in summarized.items():
        print(f"Title: '{title}'")
        tts = gTTS(text=story, lang='en')
        # Replace invalid characters in the title for file naming

        title = self.sanitize_filename(title)
        # Save the audio file
        audio_file = f"{title}.mp3"
        tts.save(audio_file)

        # Play the audio file
        display(Audio(audio_file))


##Calling A.I.N.A.

In [ ]:
%%time

if __name__ == '__main__':
  tickers = [ '^GSPC', 'MSFT', 'AAPL', 'GOOG']
  aina = AINA()
  link, story = aina.call_news_functions(2, tickers, True)
  summary = aina.summarize(story)
  aina.voice(summary)